<a href="https://colab.research.google.com/github/rspsbc/alura/blob/main/Aula05_alura_document_search_and_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install -q -U google-generativeai

In [11]:
import numpy as np
import pandas as pd
import google.generativeai as genai

api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

In [12]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [13]:
title = "Conhecendo gráficos do mercado de ações"
sample_text = ("(Título: Mercado de ações, descomplicando"
"\n"
"\n"
"Artigo completo:\n"
"\n"
"\n"
"Ações, opções, renda fixa, debêntures. Independente de onde investir, tenha certeza que seguir orientações básicas vão te ajudar a ganhar mais e perder menos)")

embeddings = genai.embed_content(model="models/embedding-001",
                                content=sample_text,
                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.002638026, -0.02645896, 0.016309474, 0.0253683, 0.051210545, 0.038372885, 0.014682282, -0.007762499, -0.025248911, 0.028504236, -0.037241343, 0.015670871, -0.04726795, 0.023317782, 0.015939534, -0.027073767, -0.0066981018, 0.008954236, 0.019901149, -0.0033209496, -0.016719485, -0.00084708486, 0.0109788785, -0.00011665014, -0.0033441375, -0.025752159, 0.006204615, -0.06444463, -0.05223119, 0.06102386, -0.059094116, 0.028956203, -0.038356513, 0.02202649, 0.04031653, -0.042604752, -0.021547647, 0.022205394, 0.0007772028, 0.01878868, -0.01387045, -0.06102346, -0.010498741, 0.0071025034, -0.022187801, 0.0048003937, 0.018275771, 0.005068043, 0.0035111061, -0.09368529, -0.027558092, -0.00940741, 0.119010404, 0.004873604, 0.048828613, -0.059305266, 0.0353468, -0.03796217, -0.03152468, -0.002622374, -0.00062835723, 0.029771958, 0.0343885, 0.014448083, 0.020923607, -0.056299813, -0.026369967, -0.007186301, 0.031454384, 0.019407187, -0.046182517, -0.061130222, 0.030084636, -0.022

In [14]:
DOCUMENT1 = {
    "Título": "Operações básicas da bolsa de valores no Brasil",
    "Conteúdo": "Ações que podem ser operadas na bolsa que tenham valores de aporte baixas, mas que podem render um valor bom, desde que operada no prazo adequado."
}

DOCUMENT2 = {
    "Título": "Gráficos",
    "Conteúdo": "Os gráficos para operação do mercado e suas variáveis."
}

DOCUMENT3 = {
    "Título": "Tempo certo de entrada ",
    "Conteúdo": "Não é achar que tem o feeling do momento, mas sim a técnica certa que vai te indicar esse momento."
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [15]:
df = pd.DataFrame(documents)
df.columns = ["Título", "Conteudo"]
df

,Título,Conteudo
0,Operações básicas da bolsa de valores no Brasil,Ações que podem ser operadas na bolsa que tenh...
1,Gráficos,Os gráficos para operação do mercado e suas va...
2,Tempo certo de entrada,"Não é achar que tem o feeling do momento, mas ..."


In [16]:
model = "models/embedding-001"

In [17]:
def embed_fn(title, text):
        return genai.embed_content(model=model, content=text, title=title, task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [18]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Título"], row["Conteudo"]), axis=1)
df

,Título,Conteudo,Embeddings
0,Operações básicas da bolsa de valores no Brasil,Ações que podem ser operadas na bolsa que tenh...,"[0.012975098, -0.026909543, -0.009728431, 0.04..."
1,Gráficos,Os gráficos para operação do mercado e suas va...,"[0.008400697, -0.042534932, -0.044081185, 0.02..."
2,Tempo certo de entrada,"Não é achar que tem o feeling do momento, mas ...","[0.034874298, -0.03974168, -0.021845043, 0.014..."


In [19]:
def gerar_e_buscar_consulta(consulta, base, model):
    embedding_da_consulta = genai.embed_content(model=model,
                                                content=consulta,
                                                task_type="RETRIEVAL_QUERY")["embedding"]

    produtos_escalares = np.dot(np.stack(df["Embeddings"]),embedding_da_consulta)

    indice = np.argmax(produtos_escalares)
    return df.iloc[indice]["Conteudo"]

In [20]:
consulta = "quais operações básicas da bolsa de valores no Brasil?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Ações que podem ser operadas na bolsa que tenham valores de aporte baixas, mas que podem render um valor bom, desde que operada no prazo adequado.


In [21]:
generation_config = {
  "temperature": 0.5,
  "candidate_count": 1
}

In [26]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)

response = model_2.generate_content(prompt)
print(response.text)

Bora investir na bolsa sem gastar um rim? Aqui vão algumas ações que podem te render uma grana legal, mesmo se você não tiver muito dinheiro pra começar. Só não esquece de operar na hora certa, viu?
